# Implementation

Author: Eduardo Miguel Firvida Donestevez

In this tutorial, you will learn how to:

- Static deformation of IEA 15MW Offshore Reference Turbine


In [2]:
!pip install --no-cache-dir -e /turbine_mesher

import os
from pathlib import Path

CURRENT_FOLDER = Path().resolve()
DATA_FOLDER = os.path.join(CURRENT_FOLDER, "data")
OUTPUT_FOLDER = os.path.join(CURRENT_FOLDER, "output")


Obtaining file:///turbine_mesher
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.8/39.8 MB 106.0 MB/s eta 0:00:00a 0:00:01
  Building editable for turbine-mesher (pyproject.toml) ... done
  Created wheel for turbine-mesher: filename=turbine_mesher-0.1.0-py3-none-any.whl size=1150 sha256=aaedd90ead3818e5de6d065933e28652b8aa19bef71af5a322a436ca0db47c74
  Stored in directory: /tmp/pip-ephem-wheel-cache-w93artrh/wheels/14/8d/6c/2dd37501f447101d7f4805b86d7e8d7ad6b33a38f48e04f51d
Successfully built turbine-mesher
  Attempting uninstall: gmsh
    Found existing installation: gmsh 4.13.1.dev1
    Not uninstalling gmsh at /usr/local/lib, outside environment /dolfinx-env
    Can't uninstall 'gmsh'. No files were found to uninstall.
  Attempting unins

## Read pyNumad blade definition

The blade definition was taken from the PyNumad Original github repositories examples
here: https://github.com/sandialabs/pyNuMAD/blob/main/examples/example_data/IEA-15-240-RWT.yaml


In [3]:
bladeYaml = os.path.join(DATA_FOLDER, "IEA-15-240-RWT.yaml")


Create mesh object, and mesh solid FEM blade


In [4]:
from turbine_mesher.mesh import Mesh

blade = Mesh(bladeYaml, element_size=0.2)


Updating station: 49 TE separation from 0.000648680513013211 to 0.001


In [ ]:
blade.solid_mesh()


Output()

Mesh statistics:
Total nodes: 156900
Total cells: 686628
Cell types: tetra
 -> tetra: 686628 Cells

### View mesh with PyVista

To plot the mesh we need to transform the MeshioMesh to pyVista.


In [6]:
import numpy as np
import pyvista as pv
from pyvista import CellType


def meshio_to_pyvista(mesh):
    points = mesh.points
    cells = []
    cell_type = np.array(
        [CellType.TETRA for i in range(sum(len(i.data) for i in mesh.cells))], np.int32
    )

    for cell_block in mesh.cells_dict.values():
        for cell in cell_block:
            cells.extend([4] + cell.tolist())

    cells = np.array(cells)
    grid = pv.UnstructuredGrid(cells, cell_type, points)
    return grid


pv_mesh = meshio_to_pyvista(blade.mesh)
plotter = pv.Plotter()
plotter.add_mesh(pv_mesh, show_edges=True)
plotter.show()


Widget(value='<iframe src="http://localhost:46113/index.html?ui=P_0x7afa45798200_0&reconnect=auto" class="pyvi…

Prepare mesh for DOLFINx


In [ ]:
import basix.ufl
import dolfinx as dfx
import ufl
from mpi4py import MPI

nodes = blade.mesh.points
connectivity = [i for i in blade.mesh.cells_dict.values()][0]
c_el = ufl.Mesh(
    basix.ufl.element("Lagrange", "tetrahedron", 1, shape=(nodes.shape[1],))
)
domain = dfx.mesh.create_mesh(MPI.COMM_SELF, connectivity, nodes, c_el)


In [ ]:
V = dfx.fem.functionspace(domain, ("Lagrange", 1, (domain.geometry.dim,)))


## Boundary conditions

As we would like to clamp the boundary at $x=0$, we do this by using a marker function, which locate the facets where $x$ is close to zero by machine precision.


In [ ]:
def clamped_boundary(x):
    return np.isclose(x[2], 0, atol=1e-3)


fdim = domain.topology.dim - 1
dim = domain.topology.dim
boundary_facets = dfx.mesh.locate_entities_boundary(domain, fdim, clamped_boundary)

u_D = np.array([0, 0, 0], dtype=dfx.default_scalar_type)
bc = dfx.fem.dirichletbc(
    u_D, dfx.fem.locate_dofs_topological(V, fdim, boundary_facets), V
)


As we want the traction $T$ over the remaining boundary to be $0$, we create a `dolfinx.Constant`


In [ ]:
ds = ufl.Measure("ds", domain=domain)
dx = ufl.Measure("dx", domain=domain)


## Variational formulation

We are now ready to create our variational formulation in close to mathematical syntax, as for the previous problems.


In [ ]:
E = dfx.fem.Constant(domain, 210e3)
nu = dfx.fem.Constant(domain, 0.3)
rho = 2e-3
g = 9.81

lmbda = E * nu / (1 + nu) / (1 - 2 * nu)
mu = E / 2 / (1 + nu)


def epsilon(v):
    return ufl.sym(ufl.grad(v))


def sigma(v):
    return lmbda * ufl.tr(epsilon(v)) * ufl.Identity(dim) + 2 * mu * epsilon(v)


u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
v = ufl.TestFunction(V)


f = dfx.fem.Constant(domain, np.array([0, 200.0, 0]))
a = ufl.inner(sigma(u), epsilon(v)) * dx
L = ufl.inner(f, v) * ds


```{note}
Note that we used `nabla_grad` and optionally `nabla_div` for the variational formulation, as oposed to our previous usage of
`div` and `grad`. This is because for scalar functions $\nabla u$ has a clear meaning
$\nabla u = \left(\frac{\partial u}{\partial x}, \frac{\partial u}{\partial y}, \frac{\partial u}{\partial z} \right)$.

However, if $u$ is vector valued, the meaning is less clear. Some sources define $\nabla u$ as a matrix with the elements $\frac{\partial u_j}{\partial x_i}$, while other  sources prefer
$\frac{\partial u_i}{\partial x_j}$. In DOLFINx `grad(u)` is defined as the matrix with elements $\frac{\partial u_i}{\partial x_j}$. However, as it is common in continuum mechanics to use the other definition, `ufl` supplies us with `nabla_grad` for this purpose.
```

## Solve the linear variational problem

As in the previous demos, we assemble the matrix and right hand side vector and use PETSc to solve our variational problem


In [ ]:
from dolfinx.fem.petsc import LinearProblem

problem = LinearProblem(
    a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"}
)
uh = problem.solve()


## Visualization


As in the previous demos, we can either use Pyvista or Paraview for visualization. We start by using Pyvista. Instead of adding scalar values to the grid, we add vectors.


In [ ]:
import pyvista as pv

p = pv.Plotter()
topology, cell_types, geometry = dfx.plot.vtk_mesh(V)
grid = pv.UnstructuredGrid(topology, cell_types, geometry)

grid["u"] = uh.x.array.reshape((geometry.shape[0], -1))
actor_0 = p.add_mesh(grid, style="wireframe")
warped = grid.warp_by_vector("u", factor=1e-8)
actor_1 = p.add_mesh(warped, show_edges=True)
p.show_axes()
p.show()


Authorization required, but no authorization protocol specified

Authorization required, but no authorization protocol specified

2024-11-27 04:34:28.072 (1719.796s) [    7FE64A667140]vtkXOpenGLRenderWindow.:1290  WARN| vtkXOpenGLRenderWindow (0x46f618b0): bad X server connection. DISPLAY=:1


Widget(value='<iframe src="http://localhost:38991/index.html?ui=P_0x7fe566ba5760_11&reconnect=auto" class="pyv…

We could also use Paraview for visualizing this.
As explained in previous sections, we save the solution with `XDMFFile`.
After opening the file `deformation.xdmf` in Paraview and pressing `Apply`, one can press the `Warp by vector button` ![Warp by vector](warp_by_vector.png) or go through the top menu (`Filters->Alphabetical->Warp by Vector`) and press `Apply`. We can also change the color of the deformed beam by changing the value in the color menu ![color](color.png) from `Solid Color` to `Deformation`.


In [ ]:
with dfx.io.XDMFFile(domain.comm, "deformation.xdmf", "w") as xdmf:
    xdmf.write_mesh(domain)
    uh.name = "Deformation"
    xdmf.write_function(uh)

with dfx.io.VTKFile(domain.comm, "deformation.pvd", "w") as xdmf:
    xdmf.write_mesh(domain)
    uh.name = "Deformation"
    xdmf.write_function(uh)


## Stress computation

As soon as the displacement is computed, we can compute various stress measures. We will compute the von Mises stress defined as $\sigma_m=\sqrt{\frac{3}{2}s:s}$ where $s$ is the deviatoric stress tensor $s(u)=\sigma(u)-\frac{1}{3}\mathrm{tr}(\sigma(u))I$.


In [ ]:
s = sigma(uh) - 1.0 / 3 * ufl.tr(sigma(uh)) * ufl.Identity(len(uh))
von_Mises = ufl.sqrt(3.0 / 2 * ufl.inner(s, s))


The `von_Mises` variable is now an expression that must be projected into an appropriate function space so that we can visualize it. As `uh` is a linear combination of first order piecewise continuous functions, the von Mises stresses will be a cell-wise constant function.


In [ ]:
V_von_mises = dfx.fem.functionspace(domain, ("DG", 0))
stress_expr = dfx.fem.Expression(von_Mises, V_von_mises.element.interpolation_points())
stresses = dfx.fem.Function(V_von_mises)
stresses.interpolate(stress_expr)


In the previous sections, we have only visualized first order Lagrangian functions. However, the Von Mises stresses are piecewise constant on each cell. Therefore, we modify our plotting routine slightly. The first thing we notice is that we now set values for each cell, which has a one to one correspondence with the degrees of freedom in the function space.


In [ ]:
warped.cell_data["VonMises"] = stresses.x.petsc_vec.array
warped.set_active_scalars("VonMises")
p = pv.Plotter()
p.add_mesh(warped)
p.show_axes()
p.show()


Authorization required, but no authorization protocol specified

Authorization required, but no authorization protocol specified

2024-11-27 04:34:34.920 (1726.643s) [    7FE64A667140]vtkXOpenGLRenderWindow.:1290  WARN| vtkXOpenGLRenderWindow (0x7ecf8810): bad X server connection. DISPLAY=:1


Widget(value='<iframe src="http://localhost:38991/index.html?ui=P_0x7fe5d8109d00_12&reconnect=auto" class="pyv…